<a href="https://colab.research.google.com/github/DahianaRH/Project_ModYSim/blob/main/99_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto: Predicción de RENDIMIENTO_GLOBAL
## Pruebas Saber Pro - UDEA/ai4eng 20252
Curso: Modelos y Simulación  
Estudiante: Sandy Ruiz Higuita

Este notebook documenta de forma estructurada el proceso de diseño, entrenamiento y evaluación de un modelo de clasificación para predecir el nivel de `RENDIMIENTO_GLOBAL` de cada observación del dataset. El desarrollo incluye:

1. Carga y exploración inicial de los datos.  
2. Preprocesamiento y codificación.  
3. Entrenamiento del modelo.  
4. Validación interna.  
5. Generación de predicciones sobre el conjunto `test`.  
6. Exportación del archivo `submission.csv` para Kaggle.


## Importación de librerías
En esta sección se cargan las librerías necesarias para el procesamiento de datos, modelado y exportación de resultados.

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
!pip install catboost
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

## Carga del conjunto de entrenamiento y prueba

A continuación se cargan los archivos `train.csv` y `test.csv` desde Google Drive, que contienen las variables predictoras y la variable objetivo. El dataset de prueba no incluye la variable objetivo, por lo que solo se usa para generar la predicción final enviada a Kaggle.


In [32]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_ModYSim_/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_ModYSim_/test.csv')

train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


#Exploración Inicial

Se realiza una inspección general de dataset para identificar el tipo de variables, la presencia de valores faltantes y la distribución de la variable objetivo. Esta etapa nos permite comprender la estructura del problema y orientar las decisiones de preprocesamiento.



In [33]:
train.info()
train.describe(include="all")
train['RENDIMIENTO_GLOBAL'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

,count
RENDIMIENTO_GLOBAL,
alto,175619
bajo,172987
medio-bajo,172275
medio-alto,171619


## Preparación de los datos

Se elimina cualquier fila con la etiqueta objetivo vacía para garantizar consistencia en el entrenamiento.

También se separa el ID (no informativo para el modelo) y se conserva únicamente para reconstruir el archivo final de predicciones.

La variable objetivo se codifica con LabelEncoder para convertir las clases nominales en valores enteros adecuados para los modelos de clasificación.

In [34]:
# Eliminación de filas con objetivo vacío
train = train.dropna(subset=['RENDIMIENTO_GLOBAL'])

# Guardar IDs del test antes de eliminar la columna
test_ids = test['ID']

# Eliminar ID en ambos datasets
train = train.drop(columns=['ID'])
test_clean = test.drop(columns=['ID'])

# Separar variables predictoras y objetivo
X = train.drop(columns=['RENDIMIENTO_GLOBAL'])
y = train['RENDIMIENTO_GLOBAL']

# Codificación de las etiquetas del objetivo
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

## División del conjunto de entrenamiento y validación

Para evaluar el desempeño del modelo, se crea una división interna del 80% para entrenamiento y 20% para validación.

Esta validación interna es importante porque nos permite seleccionar el mejor modelo sin usar aún los datos de prueba, cumpliendo la metodología correcta del curso.

In [35]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

#Identificación de variables categóricas

Se detectan las columnas categóricas (tipo object) y las numéricas. Esto permite aplicar transformaciones diferenciadas a cada tipo de variable.

Este paso es indispensable para el posterior uso del OneHotEncoder en el preprocesamiento del pipeline.

In [36]:
categorical_cols = X.select_dtypes(include=["object"]).columns
numeric_cols = X.select_dtypes(exclude=["object"]).columns

categorical_cols, numeric_cols

(Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
        'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
        'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
        'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
        'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
        'F_EDUCACIONMADRE'],
       dtype='object'),
 Index(['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3',
        'INDICADOR_4'],
       dtype='object'))

#Definición del Preprocesamiento + Modelo (Pipeline)

Se construye un ColumnTransformer que aplica OneHotEncoder a las variables categóricas y deja pasar sin modificaciones las variables numéricas.

Esto asegura una codificación consistente y reproducible en todas las etapas del pipeline, tanto en validación como en predicción final.

In [60]:
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

preprocess.fit(X_train)

X_train_trans = preprocess.transform(X_train)
X_val_trans   = preprocess.transform(X_val)
test_trans    = preprocess.transform(test_clean)

model_rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    random_state=42
)

pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', model_rf)
])

##Definición de modelos base

Se incluyen tres modelos basados en gradiente boosting (LightGBM, XGBoost y CatBoost) por su buen desempeño en datos tabulares y su capacidad para manejar relaciones no lineales.

Cada modelo se parametriza con valores razonables para evitar overfitting y asegurar estabilidad.

In [77]:
model_lgbm = LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=-1,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [62]:
model_xgb = XGBClassifier(
    n_estimators=1200,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='mlogloss',
    random_state=42
)

In [63]:
model_cat = CatBoostClassifier(
    iterations=1500,
    depth=8,
    learning_rate=0.03,
    loss_function='MultiClass',
    random_seed=42,
    verbose=200
)


## Entrenamiento del modelo

Cada modelo se entrena utilizando el conjunto de entrenamiento ya transformado.

El pipeline con Random Forest integra explícitamente el preprocesamiento, mientras que LightGBM, XGBoost y CatBoost utilizan los datos ya transformados manualmente.

Este diseño nos permite comparar modelos de distinta naturaleza bajo un marco coherente.

In [70]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE'],
      dtype='object'))])),
                ('model',
                 RandomForestClassifier(max_depth=12, n_estimators=300,
                                        random_state=42))])

In [64]:
model_lgbm.fit(X_train_trans, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.372515 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2657
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 879
[LightGBM] [Info] Start training from score -1.371993
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.391216


LGBMClassifier(colsample_bytree=0.8, learning_rate=0.03, n_estimators=1000,
               num_leaves=50, random_state=42, subsample=0.8)

In [65]:
model_xgb.fit(X_train_trans, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1200, n_jobs=None,
              num_parallel_tree=None, ...)

In [67]:
model_cat.fit(X_train_trans, y_train)

0:	learn: 1.3806247	total: 2.39s	remaining: 59m 42s
200:	learn: 1.2395055	total: 3m 7s	remaining: 20m 10s
400:	learn: 1.2230229	total: 6m 30s	remaining: 17m 50s
600:	learn: 1.2128124	total: 9m 44s	remaining: 14m 33s
800:	learn: 1.2063943	total: 12m 32s	remaining: 10m 56s
1000:	learn: 1.2013668	total: 15m 22s	remaining: 7m 39s
1200:	learn: 1.1975192	total: 18m 10s	remaining: 4m 31s
1400:	learn: 1.1943446	total: 21m	remaining: 1m 29s
1499:	learn: 1.1929735	total: 22m 22s	remaining: 0us


## Evaluación comparativa de todos los modelos

A continuación se evalúa cada modelo sobre el conjunto de validación interno utilizando dos métricas:
*   Accuracy: proporción de predicciones correctas.
*   F1 weighted: métrica robusta frente a desbalance de clases.

La comparación permite determinar cuál modelo logra el mejor compromiso entre precisión y estabilidad en las distintas clases del objetivo.

Los resultados se organizan en un DataFrame para facilitar la interpretación y selección del mejor modelo.


In [71]:
from sklearn.metrics import accuracy_score, f1_score
results = []

for name, m in [
    ("Pipeline_RF", pipeline),
    ("LightGBM", model_lgbm),
    ("XGBoost", model_xgb),
    ("CatBoost", model_cat)
]:
    print(f"\nEvaluando {name}...")

    # 1. Pipeline RF (usa X_val normal)
    if name == "Pipeline_RF":
        preds = m.predict(X_val)

    # 2. Modelos que requieren datos transformados
    elif name in ["LightGBM", "XGBoost", "CatBoost"]:
        preds = m.predict(X_val_trans)

        # CatBoost devuelve float, se convierte a int
        if name == "CatBoost":
            preds = preds.astype(int)

    # Métricas
    acc = accuracy_score(y_val, preds)
    f1 = f1_score(y_val, preds, average="weighted")

    results.append({
        "Modelo": name,
        "Accuracy": acc,
        "F1 weighted": f1
    })

# Tabla final
results_df = pd.DataFrame(results).sort_values("F1 weighted", ascending=False)
results_df


Evaluando Pipeline_RF...

Evaluando LightGBM...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



Evaluando XGBoost...

Evaluando CatBoost...


,Modelo,Accuracy,F1 weighted
1,LightGBM,0.438231,0.426416
2,XGBoost,0.436058,0.424419
3,CatBoost,0.434188,0.421565
0,Pipeline_RF,0.394274,0.366200


## Selección del mejor modelo

El modelo con mayor F1 weighted se selecciona como el modelo final.

Esta elección garantiza que el desempeño reportado no se basa en el conjunto de test, preservando así la integridad del proceso de validación establecido en el curso.

In [72]:
best_model_name = results_df.iloc[0]["Modelo"]
best_model_name

'LightGBM'

In [73]:
model_dict = {
    "Pipeline_RF": pipeline,
    "LightGBM": model_lgbm,
    "XGBoost": model_xgb,
    "CatBoost": model_cat
}

best_model = model_dict[best_model_name]
best_model

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.03, n_estimators=1000,
               num_leaves=50, random_state=42, subsample=0.8)

## Predicción sobre el conjunto de prueba

El modelo seleccionado se utiliza para generar predicciones sobre el dataset test.
Las predicciones numéricas se devuelven a su representación original mediante el LabelEncoder, recuperando las etiquetas:
- bajo  
- medio-bajo  
- medio-alto  
- alto

Posteriormente se genera el archivo submission.csv con el formato requerido por Kaggle.

In [75]:
# Predicción final
test_pred = best_model.predict(test_trans)

# Convertir valores numéricos a etiquetas reales
test_pred_labels = label_encoder.inverse_transform(test_pred)

submission = pd.DataFrame({
    "ID": test_ids,
    "RENDIMIENTO_GLOBAL": test_pred_labels
})

submission.to_csv("submission.csv", index=False)
submission.head()

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo


#Descarga del archivo

In [76]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Conclusiones

El notebook implementa un flujo reproducible y completo: exploración, preprocesamiento, entrenamiento, selección de modelo y generación del archivo final.

Esta estructura permite asegurar que el modelo final se selecciona de forma objetiva y sin fugas de información, manteniendo la rigurosidad del proceso de ciencia de datos.

En el video se describen brevemente estas etapas, resaltando las decisiones de diseño y la importancia de mantener una separación clara entre entrenamiento, validación y prueba.